Azure AI speech to text \
https://learn.microsoft.com/en-us/azure/ai-services/speech-service/get-started-stt-diarization?tabs=windows&pivots=programming-language-python \
https://github.com/Azure-Samples/cognitive-services-speech-sdk/tree/master

In [7]:
# continuous-recognition speech to text
import os
import time
import azure.cognitiveservices.speech as speechsdk

def conversation_transcriber_recognition_canceled_cb(evt: speechsdk.SessionEventArgs):
    print('Canceled event')

def conversation_transcriber_session_stopped_cb(evt: speechsdk.SessionEventArgs):
    print('SessionStopped event')

def conversation_transcriber_transcribed_cb(evt: speechsdk.SpeechRecognitionEventArgs):
    print('TRANSCRIBED:')
    if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print('\tText={}'.format(evt.result.text))
        print('\tSpeaker ID={}'.format(evt.result.speaker_id))
        f.write(f'{evt.result.text}\n')
    elif evt.result.reason == speechsdk.ResultReason.NoMatch:
        print('\tNOMATCH: Speech could not be TRANSCRIBED: {}'.format(evt.result.no_match_details))

def conversation_transcriber_session_started_cb(evt: speechsdk.SessionEventArgs):
    print('SessionStarted event')

def recognize_from_file(audio_file_path):
    # This example requires environment variables named "AZURE_SPEECH_KEY" and "AZURE_SPEECH_REGION"
    speech_config = speechsdk.SpeechConfig(subscription=os.environ.get('AZURE_SPEECH_KEY'), region=os.environ.get('AZURE_SPEECH_REGION'))
    speech_config.speech_recognition_language="en-US"
    
    audio_config = speechsdk.audio.AudioConfig(filename=audio_file_path)
    conversation_transcriber = speechsdk.transcription.ConversationTranscriber(speech_config=speech_config, audio_config=audio_config)

    transcribing_stop = False

    def stop_cb(evt: speechsdk.SessionEventArgs):
        #"""callback that signals to stop continuous recognition upon receiving an event `evt`"""
        print('CLOSING on {}'.format(evt))
        nonlocal transcribing_stop
        transcribing_stop = True

    # Connect callbacks to the events fired by the conversation transcriber
    conversation_transcriber.transcribed.connect(conversation_transcriber_transcribed_cb)
    conversation_transcriber.session_started.connect(conversation_transcriber_session_started_cb)
    conversation_transcriber.session_stopped.connect(conversation_transcriber_session_stopped_cb)
    conversation_transcriber.canceled.connect(conversation_transcriber_recognition_canceled_cb)
    # stop transcribing on either session stopped or canceled events
    conversation_transcriber.session_stopped.connect(stop_cb)
    conversation_transcriber.canceled.connect(stop_cb)

    conversation_transcriber.start_transcribing_async()

    # Waits for completion.
    while not transcribing_stop:
        time.sleep(.5)

    conversation_transcriber.stop_transcribing_async()

# Main
if __name__ == "__main__":
    # audio_file_path = "..\\data\\speech\\BillGates_2010.wav" 
    audio_file_path = "..\\..\\data\\speech\\yearwiththesaints_00_anonymous.wav" 

    base_path, _ = os.path.splitext(audio_file_path)
    output_file_path = base_path + ".txt"

    f = open(output_file_path, "w")

    try:
        recognize_from_file(audio_file_path)
    except Exception as err:
        print("Encountered exception. {}".format(err))
    finally:
        f.close()

    

SessionStarted event
TRANSCRIBED:
	Text=Preface of a Year with the Saints translated from the Italian by Member of the Order of Mercy. This is a Librivox recording. All Librivox recordings are in the public domain. For more information or to volunteer, please visit librivox.org. Recording by Maria Therese. A Year with the Saints.
	Speaker ID=Guest-1
TRANSCRIBED:
	Text=Translators preface.
	Speaker ID=Guest-1
TRANSCRIBED:
	Text=And presenting this work for the first time in English dress, the translator would take the opportunity of drawing the attention of the devout American reader to one or two peculiarities in its structure, which have already won for it a wide recognition among readers of books of its class in Europe.
	Speaker ID=Guest-1
TRANSCRIBED:
	Text=In the first place, the allotment of separate months to individual virtues agrees well with the practice of self examination, much recommended by many spiritual writers, who advise us to take different virtues in turn as the obje

In [8]:
# using document summarization
import os
def sample_abstractive_summarization(document) -> None:
    # [START abstract_summary]
    import os
    from azure.core.credentials import AzureKeyCredential
    from azure.ai.textanalytics import TextAnalyticsClient

    endpoint = os.environ["AZURE_LANGUAGE_ENDPOINT"]
    key = os.environ["AZURE_LANGUAGE_KEY"]

    text_analytics_client = TextAnalyticsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(key),
    )

    poller = text_analytics_client.begin_abstract_summary(document)
    abstract_summary_results = poller.result()
    for result in abstract_summary_results:
        if result.kind == "AbstractiveSummarization":
            print("Summaries abstracted:")
            [print(f"{summary.text}\n") for summary in result.summaries]
        elif result.is_error is True:
            print("...Is an error with code '{}' and message '{}'".format(
                result.error.code, result.error.message
            ))
    # [END abstract_summary]


if __name__ == "__main__":
    output_file_path = "..\\..\\data\\speech\\yearwiththesaints_00_anonymous.txt" 
    if os.path.exists(output_file_path):
        with open(output_file_path, 'r') as file:
            lines = [line.strip('\n') for line in file.readlines()]
        separator = ', '
        result = [separator.join(lines)]
        sample_abstractive_summarization(result)
    else:
        print(f"The file '{output_file_path}' does not exist, please execute continuous-recognition speech to text!")    


Summaries abstracted:
The document is a preliminary introduction to the English translation of "A Year with the Saints," a book by Maria Therese. The translation, made by a Member of the Order of Mercy, highlights the book's unique structure, which has been well-received in Europe. The book is divided into separate months for individual Virtues, which aligns with the practice of self-examination recommended by spiritual writers. It also includes daily lessons followed by action-oriented examples to illustrate the application of principles in daily life. The translator hopes that this book will be a success in the American spiritual community.



### OpenAI Speech to Text

In [6]:
# step 1 Speech to text
from openai import OpenAI
client = OpenAI()
audio_file_path = "..\\..\\data\\speech\\yearwiththesaints_00_anonymous.wav" 
audio_file= open(audio_file_path, "rb")
transcript = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file,
  response_format="text"
)

print(f"transcript: {transcript}")

# step 2 Summarize transcript
user_prompt = f"Could you please summarize the transcript ### {transcript} ###"
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": user_prompt},
  ]
)

print(f"summary: {response.choices[0].message.content}")

transcript: PREFACE OF A YEAR WITH THE SAINTS. In presenting this work for the first time in English dress, the translator would take the opportunity of drawing the attention of the devout American reader to one or two peculiarities in its structure, which have already won for it a wide recognition among readers of books of its class in Europe. In the first place, the allotment of separate months to individual virtues agrees well with the practice of self-examination much recommended by many spiritual writers, who advise us to take different virtues in turn as the object of special effort. Nothing certainly could be better fitted to assist us in the acquisition of a virtue than these selections from the writings of the most eminent masters of spiritual science, which portray its beauties, declare its necessity, or explain its various degree and the methods of its attainment. In the second place, the few words of exhortation or instruction which open the reading for each day are followe